In [289]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.models import model_from_json
import numpy as np
import pandas as pd

In [331]:
train_dir = 'Train'

val_dir = 'Validation'

test_dir = 'Test'

img_width, img_height = 28, 28
input_shape = (img_width, img_height, 3)

epochs = 30

batch_size = 16

nb_train_samples = 11120
nb_validation_samples = 1656
nb_test_samples = 1152


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(8))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

Found 10868 images belonging to 8 classes.


In [332]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

Found 1656 images belonging to 8 classes.


In [333]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

Found 1152 images belonging to 8 classes.


In [334]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/30
695/695 [==============================] - 130s 188ms/step - loss: 2.0443 - acc: 0.1978 - val_loss: 1.9410 - val_acc: 0.3174
Epoch 2/30
695/695 [==============================] - 27s 39ms/step - loss: 1.6310 - acc: 0.3784 - val_loss: 1.1900 - val_acc: 0.6141
Epoch 3/30
695/695 [==============================] - 27s 39ms/step - loss: 1.1360 - acc: 0.5846 - val_loss: 0.9750 - val_acc: 0.6650
Epoch 4/30
695/695 [==============================] - 27s 39ms/step - loss: 0.8726 - acc: 0.6808 - val_loss: 0.8892 - val_acc: 0.6857
Epoch 5/30
695/695 [==============================] - 28s 41ms/step - loss: 0.7164 - acc: 0.7380 - val_loss: 0.8714 - val_acc: 0.7306
Epoch 6/30
695/695 [==============================] - 26s 38ms/step - loss: 0.6058 - acc: 0.7759 - val_loss: 0.7448 - val_acc: 0.7403
Epoch 7/30
695/695 [==============================] - 26s 38ms/step - loss: 0.5277 - acc: 0.8049 - val_loss: 0.8860 - val_acc: 0.7294
Epoch 8/30
695/695 [==============================] - 25s 36

In [335]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

Аккуратность на тестовых данных: 74.39%


In [336]:
model_json = model.to_json()
# Записываем модель в файл
json_file = open("mnist_model.json", "w")
json_file.write(model_json)
json_file.close()

model_yaml = model.to_yaml()
yaml_file = open("mnist_model.yml", "w")
# Записываем модель в файл
yaml_file.write(model_yaml)
yaml_file.close()
model.save_weights("mnist_model.h5")

In [337]:
json_file = open("mnist_model.json", "r")
loaded_model_json = json_file.read()
json_file.close()
# Создаем модель на основе загруженных данных
loaded_model = model_from_json(loaded_model_json)
# Загружаем веса в модель
loaded_model.load_weights("mnist_model.h5")
loaded_model.compile(loss="sparse_categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])
# Проверяем модель на тестовых данных
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Точность модели на тестовых данных: %.2f%%" % (scores[1]*100))

Точность модели на тестовых данных: 74.39%


In [383]:
test_dir = 'ts'
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse')

Found 1 images belonging to 1 classes.


In [384]:
test_generator.reset()
predict = model.predict_generator(test_generator, steps = None, max_queue_size=10, workers=1, use_multiprocessing=False, verbose=0)
for i in predict:
    print(np.argmax(i))



2
